In [0]:
CREATE OR REPLACE TABLE customer_dim (
  customer_id BIGINT,
  region STRING,
  total_amount BIGINT
);

INSERT INTO customer_dim VALUES
  (101, 'US_WEST', 20),
  (102, 'US_EAST', 10),
  (103,  'US_WEST', 15);
  
DROP TEMPORARY TABLE IF EXISTS temp_customers;

In [0]:
CREATE TEMPORARY TABLE temp_customers
(customer_id BIGINT,
region STRING,
total_amount BIGINT);

INSERT INTO temp_customers
SELECT customer_id, region, total_amount
FROM customer_dim
WHERE region = 'US_WEST';

UPDATE temp_customers
SET total_amount = total_amount * 0.90
WHERE customer_id = 101;


In [0]:
SELECT * FROM temp_customers;

In [0]:
DESCRIBE TABLE EXTENDED temp_customers;

In [0]:
CREATE TEMPORARY VIEW temp_customers_v AS
SELECT
  customer_id,
  region,
  CASE
    WHEN customer_id = 101 THEN total_amount * 0.90
    ELSE total_amount
  END AS total_amount
FROM customer_dim
WHERE region = 'US_WEST';

In [0]:
CREATE OR REPLACE PROCEDURE sp_refresh()
LANGUAGE SQL
SQL SECURITY INVOKER
AS BEGIN

  CREATE TEMPORARY TABLE tmp AS
  SELECT id, v*2 AS v2 FROM src;

  MERGE INTO tgt t
  USING tmp s
  ON t.id = s.id
  WHEN MATCHED THEN UPDATE SET t.v2 = s.v2
  WHEN NOT MATCHED THEN INSERT (id, v2) VALUES (s.id, s.v2);
END;

In [0]:
CALL sp_refresh();

In [0]:
SELECT * FROM tgt;